In [1]:
%pip install sktime==0.11.3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import timeit
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import fbeta_score, make_scorer
import tensorflow as tf
from pickle import dump
from sklearn.model_selection import KFold


#Import Models
from sktime.classification.hybrid import HIVECOTEV2
from sktime.classification.hybrid import HIVECOTEV1
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import RandomIntervalSpectralEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime._contrib.vector_classifiers._rotation_forest import RotationForest

from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier


from sktime.datasets import load_from_arff_to_dataframe as load_arff
#numpy.version.version

# Rocket Classifier code

In [3]:
%pip install tsai

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: imbalanced-learn>=0.8.0 in d:\pessoais\anacondafiles\envs\teste_with_gpu\lib\site-packages (from tsai) (0.9.0)



In [4]:
#%pip install sklearn --upgrade

In [5]:
import sklearn
from sklearn.linear_model import RidgeClassifierCV, RidgeCV
from sklearn.metrics import make_scorer
from tsai.imports import *
from tsai.data.external import *
from tsai.models.layers import *

D:\Pessoais\AnacondaFiles\envs\teste_with_gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
X, y = load_arff("AllCandidas_TRAIN_V2.arff")
X

,dim_0
0,0 23210.0 1 50050.0 2 18240.0 3 38460.0 4 306.8 5 1185.0 6 2968.0 7 2.0 dtype: float64
1,0 14280.0 1 57150.0 2 14070.0 3 30840.0 4 193.9 5 679.4 6 2026.0 7 1.0 dtype: float64
2,0 19770.0 1 47210.0 2 17700.0 3 17100.0 4 255.3 5 797.0 6 2547.0 7 2.0 dtype: float64
3,0 47070.0 1 23320.0 2 28060.0 3 24090.0 4 627.3 5 2421.0 6 5485.0 7 2.0 dtype: float64
4,0 7782.0 1 54440.0 2 9657.0 3 16690.0 4 134.5 5 482.9 6 1370.0 7 1.0 dtype: float64
...,...
12251,0 12790.0 1 59740.0 2 12670.0 3 37410.0 4 159.2 5 682.5 6 1871.0 7 1.0 dtype: float64
12252,0 14440.0 1 54450.0 2 14340.0 3 20840.0 4 226.0 5 662.7 6 2043.0 7 2.0 dtype: float64
12253,0 80170.0 1 7178.0 2 13460.0 3 37790.0 4 1968.0 5 12640.0 6 7661.0 7 1.0 dtype: float64
12254,0 12610.0 1 63570.0 2 11460.0 3 32130.0 4 164.4 5 513.9 6 1593.0 7 1.0 dtype: float64


In [7]:
#export
class RocketClassifier(sklearn.pipeline.Pipeline):
    """Time series classification using ROCKET features and a linear classifier"""
    
    def __init__(self, num_kernels=10_000, normalize_input=True, random_state=None, 
                 alphas=np.logspace(-3, 3, 7), normalize_features=True, memory=None, verbose=False, scoring=None, class_weight=None, **kwargs):
        """
        RocketClassifier is recommended for up to 10k time series. 
        For a larger dataset, you can use ROCKET (in Pytorch).
        scoring = None --> defaults to accuracy.
        
        Rocket args:            
            num_kernels     : int, number of random convolutional kernels (default 10,000)
            normalize_input : boolean, whether or not to normalise the input time series per instance (default True)
            random_state    : Optional random seed (default None)

        """
        try: 
            import sktime
            from sktime.transformations.panel.rocket import Rocket
        except ImportError:
            print("You need to install sktime to be able to use RocketClassifier")
            
        self.steps = [('rocket', Rocket(num_kernels=num_kernels, normalise=normalize_input, random_state=random_state)),
                      ('ridgeclassifiercv', RidgeClassifierCV(alphas=alphas, normalize=normalize_features, scoring=scoring, 
                                                              class_weight=class_weight, **kwargs))]
        store_attr()
        self._validate_steps()

    def __repr__(self):  
        return f'Pipeline(steps={self.steps.copy()})'

    def save(self, fname='Rocket', path='./models'):
        path = Path(path)
        filename = path/fname
        with open(f'{filename}.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)

In [8]:
#export
def load_rocket(fname='Rocket', path='./models'):
    path = Path(path)
    filename = path/fname
    with open(f'{filename}.pkl', 'rb') as input:
        output = pickle.load(input)
    return output

In [9]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [10]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [11]:
def create_dt_metrics():
#     columns = ['KNeighborsTimeSeriesClassifier','ShapeletTransformClassifier',
#                                     'TimeSeriesForestClassifier','HIVECOTE1','HIVECOTE2','RISE',
#                                     'ROCKET','BOSSEnsemble']
    columns = ['BOSSEnsemble']
    df_precision = pd.DataFrame(data= None,
                           columns=columns)
    df_accuracy  = pd.DataFrame(data= None,
                           columns=columns)
    df_f1        = pd.DataFrame(data= None,
                           columns=columns)
    df_recall    = pd.DataFrame(data= None,
                           columns=columns)
    df_std       = pd.DataFrame(data= None,
                           columns=columns)
    return df_precision, df_accuracy, df_f1, df_recall, df_std

In [12]:
def populate_df_metrics(precision, accuracy, f1, recall):
    print(f"Precision in populate_df_metrics: {precision}")
    df_precision = df_precision.append(precision, ignore_index=True)
    print(f"Df precision after append: {df_precision}")
    df_accuracy.append(pd.DataFrame(accuracy))
    df_f1.append(pd.DataFrame(f1))
    df_recall.append(pd.DataFrame(recall))
    #return df_precision, df_accuracy, df_f1, df_recall
  
def save_std(df_accuracy,df_std):
    for row in df_accuracy:
        df_std[row] = np.std(df_accuracy[row])
        print(f"STD COL: {df_std[row]}")
    print(f"STD antes do %save: {df_std}")
    df_std.to_csv("results2/" + 'df_all_std.csv', index=False)
    print(f"Save STD: {df_std}")

In [13]:
def create_models(models):
#     models.append(('KNeighborsTimeSeriesClassifier',KNeighborsTimeSeriesClassifier()))
#     models.append(('ShapeletTransformClassifier',ShapeletTransformClassifier(
#         estimator=RotationForest(n_estimators=6),
#         n_shapelet_samples=500,
#         max_shapelets=20,
#         batch_size=100,
#     )))
#     models.append(('TimeSeriesForestClassifier',TimeSeriesForestClassifier()))
#     models.append(('HIVECOTE1',HIVECOTEV1(
#                 stc_params={
#                     "estimator": RotationForest(n_estimators=3),
#                     "n_shapelet_samples": 100,
#                     "max_shapelets": 10,
#                     "batch_size": 20,
#                 },
#                 tsf_params={"n_estimators": 3},
#                 rise_params={"n_estimators": 3},
#                 cboss_params={"n_parameter_samples":1, "max_ensemble_size": 1, "min_window":1, "max_win_len_prop":1},
#                 verbose=10
#             )))
#     models.append(('HIVECOTE2',HIVECOTEV2(
#         stc_params={
#             "estimator": RotationForest(n_estimators=3),
#             "n_shapelet_samples": 100,
#             "max_shapelets": 10,
#             "batch_size": 20,
#         },
#         drcif_params={"n_estimators": 2, "n_intervals": 2, "att_subsample_size": 2},
#         arsenal_params={"num_kernels": 50, "n_estimators": 3},
#         tde_params={
#             "n_parameter_samples": 10,
#             "max_ensemble_size": 3,
#             "randomly_selected_params": 5,
#             "max_win_len_prop":2,
#             "min_window":1,
#         },
#         verbose=10
#     )))
#     models.append(('RISE',RandomIntervalSpectralEnsemble()))
#     models.append(('ROCKET',RocketClassifier()))
    models.append(('BOSSEnsemble',BOSSEnsemble(max_ensemble_size=3,min_window =1, max_win_len_prop=2)))
    return models

In [14]:
#save metrics
def save_results(df_precision, df_accuracy, df_f1, df_recall, models):
    #save the model
    for name, model in models:
        print("Saving model...")
        dump(model, open('results2/'+name+'_model.pkl', 'wb'))
        print("Saving metrics...")
        print(f"Df accuracy: {df_accuracy}")
        df_precision.to_csv("results2/" + 'df_'+name+'_precision.csv', index=False)
        df_accuracy.to_csv("results2/" + 'df_'+name+'_accuracy.csv', index=False)
        df_f1.to_csv("results2/" + 'df_'+name+'_f1.csv', index=False)
        df_recall.to_csv("results2/" + 'df_'+name+'_recall.csv', index=False)
    print("Accuracy of models:")
    print(df_accuracy)

### New code

In [18]:
def treine_model(model, name):
    print(f"Treining {name}...")
    print("In trainning...")
    model.fit(X_train, y_train)
    print("Finished training...")
    df_train = cross_validate(model, X_train, y_train, cv=kf, scoring=scoring,return_train_score=True)
    print(df_train.keys())
    save_results(pd.DataFrame(df_train),name, model,'train')
    print("***********")
    print("score val")
    print()
    df_validation = cross_validate(model, X_val, y_val, cv=kf, scoring=scoring)
    print(df_validation.keys())
    save_results(pd.DataFrame(df_validation), name, model,'val')
    return df_train, df_validation

In [19]:
import copy
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)
X, y = load_arff("AllCandidas_TRAIN_V3.arff")
#X_train1, X_val, y_train1, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
models =  []
models = create_models(models)
def gpu(X, y, kf, models):
    scoring = ['accuracy','precision_macro', 'recall_macro','f1_macro']
    df_precision, df_accuracy, df_f1, df_recall, df_std = create_dt_metrics()
    with tf.device('/device:GPU:0'):
        i= 0
        df_precision, df_accuracy, df_f1, df_recall, df_std = create_dt_metrics()
        print(f"Quantidade de modelos{len(models)}")
        df_train, df_validation = [treine_model(model, name) for name, model in models]


In [20]:
 gpu(X_train,y_train,kf,models)

Quantidade de modelos1
Treining BOSSEnsemble...
In trainning...


ValueError: could not broadcast input array from shape (4,) into shape (9,)

In [14]:
import copy
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
X, y = load_arff("AllCandidas_TRAIN_V2.arff")
print(np.shape(X))
print(type(X))
precision = []
accuracy = []
f1 = []
recall = []

def gpu(X, y, kf):
    models =  []
    models = create_models(models)
    with tf.device('/device:GPU:0'):
        i= 0
        df_precision, df_accuracy, df_f1, df_recall, df_std = create_dt_metrics()
        generic_dict =  {'BOSSEnsemble':[]}
        precision = copy.deepcopy(generic_dict)
        accuracy  = copy.deepcopy(generic_dict)
        f1        = copy.deepcopy(generic_dict)
        recall    = copy.deepcopy(generic_dict)
        for train_index, test_index in kf.split(X):
            X_train, X_test =  X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y[train_index], y[test_index]
            #print(f"X_test type:{type(X_test)}")
            #print(f"X_test type:{X_test}")

#             generic_dict =  {'KNeighborsTimeSeriesClassifier':[],'ShapeletTransformClassifier':[],
#                     'TimeSeriesForestClassifier':[],'HIVECOTE1':[],'HIVECOTE2':[],'RISE':[],
#                     'ROCKET':[],'BOSSEnsemble':[]}

            print(f"Quantidade de modelos{len(models)}")
            for name, model in models:
                print(f"Treining {name}...")
                print(f"Interaction {i}")
                print("In trainning...")
                model.fit(X_train, y_train)
                print("In prediction...")
                predictions = model.predict(X_test)
                print("Finished prediction!")
                precision[name].append(precision_score(y_test, predictions, average='macro',zero_division=1))
                accuracy[name].append(accuracy_score(y_test, predictions))
                recall[name].append(recall_score(y_test, predictions, average='macro',zero_division=1))
                f1[name].append(f1_score(y_test, predictions, average='macro'))
            i=i+1
            print("\n\n")
            print(f"Precision: {precision}, \n Accuracy:{accuracy}, \n Recall: {recall}, \n F1: {f1}")
            df_precision = pd.DataFrame(precision)
            df_accuracy  = pd.DataFrame(accuracy)
            df_f1        = pd.DataFrame(f1)
            df_recall    = pd.DataFrame(recall)
            save_results(df_precision, df_accuracy, df_f1, df_recall, models)
            save_std(df_accuracy, df_std)


(12256, 1)
<class 'pandas.core.frame.DataFrame'>


In [15]:
if tf.config.list_physical_devices('GPU'):
    print("Using MirroredStrategy")
    strategy = tf.distribute.MirroredStrategy()
else:  # Use the Default Strategy
    print("Using Default Strategy")
    strategy = tf.distribute.get_strategy()
with strategy.scope():
    # Do something interesting
    print(tf.Variable(1.))
    gpu(X,y,kf)

Using MirroredStrategy
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
MirroredVariable:{
  0: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
}
Quantidade de modelos1
Treining BOSSEnsemble...
Interaction 0
In trainning...
In prediction...
Finished prediction!



Precision: {'BOSSEnsemble': [0.718218604525774]}, 
 Accuracy:{'BOSSEnsemble': [0.7169657422512234]}, 
 Recall: {'BOSSEnsemble': [0.7177608403777439]}, 
 F1: {'BOSSEnsemble': [0.7153801198646205]}
Saving model...
Saving metrics...
Df accuracy:    BOSSEnsemble
0      0.716966
Accuracy of models:
   BOSSEnsemble
0      0.716966
STD COL: Series([], Name: BOSSEnsemble, dtype: float64)
STD antes do %save: Empty DataFrame
Columns: [BOSSEnsemble]
Index: []
Save STD: Empty DataFrame
Columns: [BOSSEnsemble]
Index: []
Quantidade de modelos1
Treining BOSSEnsemble...
Interaction 1
In trainning...
In prediction...
Finished prediction!



Precision: {'BOSSEnsemble': [0.718218

In prediction...
Finished prediction!



Precision: {'BOSSEnsemble': [0.718218604525774, 0.6553565569436118, 0.6591096348724286, 0.6613973445681727, 0.6855059809986179, 0.6844056763947832, 0.6708810997396343, 0.69641887801447, 0.7001819645575066]}, 
 Accuracy:{'BOSSEnsemble': [0.7169657422512234, 0.6500815660685155, 0.6574225122349103, 0.6623164763458401, 0.6786296900489397, 0.6810766721044046, 0.6710204081632654, 0.6906122448979591, 0.6979591836734694]}, 
 Recall: {'BOSSEnsemble': [0.7177608403777439, 0.649971135599541, 0.6590335924087358, 0.6618092772527339, 0.6848611749525513, 0.6817247856022282, 0.6682453247495324, 0.694726374678868, 0.6984398290345187]}, 
 F1: {'BOSSEnsemble': [0.7153801198646205, 0.6470729905808471, 0.6576096734002079, 0.6592909784917625, 0.6833069252730724, 0.6821968417107588, 0.6643636556200008, 0.69262931298575, 0.6963143922333034]}
Saving model...
Saving metrics...
Df accuracy:    BOSSEnsemble
0      0.716966
1      0.650082
2      0.657423
3      0.662316
4 